In [ ]:
# ! wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

In [ ]:
# ! ls

In [ ]:
# ! apt install unzip
# ! unzip cornell_movie_dialogs_corpus.zip

In [ ]:
# ! ls

In [ ]:
# ! wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

In [ ]:
# ! ls

In [ ]:
# !tar -xvf  'download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz' -C 'data'

In [ ]:
! wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz

--2020-06-19 16:49:00--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>]   3.41G  8.97MB/s    in 6m 33s  

2020-06-19 16:55:35 (8.89 MB/s) - ‘en.txt.gz’ saved [3663376519/3663376519]



In [ ]:
! apt install gunzip
! gunzip en.txt.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package gunzip


In [ ]:
! mkdir lines

In [ ]:
! split -a 3 -l 100000 --verbose en.txt lines/lines-

creating file 'lines/lines-aaa'
creating file 'lines/lines-aab'
creating file 'lines/lines-aac'
creating file 'lines/lines-aad'
creating file 'lines/lines-aae'
creating file 'lines/lines-aaf'
creating file 'lines/lines-aag'
creating file 'lines/lines-aah'
creating file 'lines/lines-aai'
creating file 'lines/lines-aaj'
creating file 'lines/lines-aak'
creating file 'lines/lines-aal'
creating file 'lines/lines-aam'
creating file 'lines/lines-aan'
creating file 'lines/lines-aao'
creating file 'lines/lines-aap'
creating file 'lines/lines-aaq'
creating file 'lines/lines-aar'
creating file 'lines/lines-aas'
creating file 'lines/lines-aat'
creating file 'lines/lines-aau'
creating file 'lines/lines-aav'
creating file 'lines/lines-aaw'
creating file 'lines/lines-aax'
creating file 'lines/lines-aay'
creating file 'lines/lines-aaz'
creating file 'lines/lines-aba'
creating file 'lines/lines-abb'
creating file 'lines/lines-abc'
creating file 'lines/lines-abd'
creating file 'lines/lines-abe'
creating

In [ ]:
# % cd ..

/content


In [ ]:
# ! wget https://github.com/PolyAI-LDN/conversational-datasets/blob/master/opensubtitles/create_data.py

--2020-06-19 13:07:40--  https://github.com/PolyAI-LDN/conversational-datasets/blob/master/opensubtitles/create_data.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘create_data.py’

create_data.py          [  <=>               ] 169.63K   321KB/s    in 0.5s    

2020-06-19 13:07:42 (321 KB/s) - ‘create_data.py’ saved [173701]



In [ ]:
import os
import re
from os import path
import tensorflow as tf
import numpy as np

In [ ]:
def _should_skip(line, min_length, max_length):
    """Whether a line should be skipped depending on the length."""
    return len(line) < min_length or len(line) > max_length

In [ ]:
def create_example(previous_lines, line, file_id):
    """Creates examples with multi-line context
    The examples will include:
        file_id: the name of the file where these lines were obtained.
        response: the current line text
        context: the previous line text
        context/0: 2 lines before
        context/1: 3 lines before, etc.
    """

    example = {
        'file_id': file_id,
        'context': previous_lines[-1],
        'response': line,
    }
    example['file_id'] = file_id
    print(file_id)
    example['context'] = previous_lines[-1]
    print(previous_lines[-1])
    extra_contexts = previous_lines[:-1]
    example.update({
        'context/{}'.format(i): context
        for i, context in enumerate(extra_contexts[::-1])
    })

    return example

In [ ]:
def _preprocess_line(line):
    line = line.decode("utf-8")

    # Remove the first word if it is followed by colon (speaker names)
    # NOTE: this wont work if the speaker's name has more than one word
    line = re.sub('(?:^|(?:[.!?]\\s))(\\w+):', "", line)

    # Remove anything between brackets (corresponds to acoustic events).
    line = re.sub("[\\[(](.*?)[\\])]", "", line)

    # Strip blanks hyphens and line breaks
    line = line.strip(" -\n")

    return line

In [ ]:
import itertools
import collections
import numpy as np
import nltk

SENT_START_TOKEN = "SENTENCE_START"
SENT_END_TOKEN = "SENTENCE_END"
UNKNOWN_TOKEN = "UNKNOWN_TOKEN"
PADDING_TOKEN = "PADDING"


def tokenize_text(text_lines):
    """
    Split text into sentences, append start and end tokens to each and tokenize
    :param text_lines: list of text lines or list of length one containing all text
    :return: list of sentences
    """
    sentences = itertools.chain(*[nltk.sent_tokenize(line.lower()) for line in text_lines])
    sentences = ["{} {} {}".format(SENT_START_TOKEN, x, SENT_END_TOKEN) for x in sentences]
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
    return tokenized_sentences

In [ ]:
def _create_examples_from_file(file_name,file_id, min_length=0, max_length=24,
                               num_extra_contexts=2):
   # _, file_id = path.split(file_name)
    #print(file_id,"#")
    previous_lines = []
    for line in open(file_name,'rb'):
        line = _preprocess_line(line)
        
        if not line:
            continue

        should_skip = _should_skip(
            line,
            min_length=min_length,
            max_length=max_length)

        if previous_lines:
            should_skip |= _should_skip(
                previous_lines[-1],
                min_length=min_length,
                max_length=max_length)

            if not should_skip:
                yield create_example(previous_lines, line, file_id)

        previous_lines.append(line)
        if len(previous_lines) > num_extra_contexts + 1:
            del previous_lines[0]

In [ ]:
def _features_to_serialized_tf_example(features):
    """Convert a string dict to a serialized TF example.
    The dictionary maps feature names (strings) to feature values (strings).
    """
    #print("hello")
    example = tf.train.Example()
    for feature_name, feature_value in features.items():
        example.features.feature[feature_name].bytes_list.value.append(
            feature_value.encode("utf-8"))
    return example.SerializeToString()


def _shuffle_examples(examples):
    examples |= ("add random key" >> beam.Map(
        lambda example: (uuid.uuid4(), example)))
    examples |= ("group by key" >> beam.GroupByKey())
    examples |= ("get shuffled values" >> beam.FlatMap(lambda t: t[1]))
    return examples

In [ ]:
# % cd ..

In [ ]:
# ! ls

In [ ]:
# ! pwd

In [ ]:
# ! ( head -1000000 en.txt ; ) > million.txt

In [ ]:
test=_create_examples_from_file(file_name='lines-aaa',file_id='lines-aaa')

In [ ]:
for x in test:
  print(x)

Streaming output truncated to the last 5000 lines.
{'file_id': 'lines-aaa', 'context': 'Are you sure?', 'response': "Sure you're not lying?", 'context/0': "That's fair enough.", 'context/1': 'Look, I want my share of the gold too.'}
lines-aaa
Sure you're not lying?
{'file_id': 'lines-aaa', 'context': "Sure you're not lying?", 'response': 'No.', 'context/0': 'Are you sure?', 'context/1': "That's fair enough."}
lines-aaa
No.
{'file_id': 'lines-aaa', 'context': 'No.', 'response': 'About the gold?', 'context/0': "Sure you're not lying?", 'context/1': 'Are you sure?'}
lines-aaa
About the gold?
{'file_id': 'lines-aaa', 'context': 'About the gold?', 'response': 'No.', 'context/0': 'No.', 'context/1': "Sure you're not lying?"}
lines-aaa
No.
{'file_id': 'lines-aaa', 'context': 'No.', 'response': 'Or the women?', 'context/0': 'About the gold?', 'context/1': 'No.'}
lines-aaa
Or the women?
{'file_id': 'lines-aaa', 'context': 'Or the women?', 'response': 'No, I...', 'context/0': 'No.', 'context/1':

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2020-06-19 15:45:55--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-19 15:45:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-19 15:45:55--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
glove_dir = './'

embeddings_index = {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

FileNotFoundError: ignored

In [ ]:
embedding_dim = 100
num_words= 72827
embedding_matrix = np.zeros((num_words, embedding_dim)) #create an array of zeros with word_num rows and embedding_dim columns
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < num_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

NameError: ignored

In [ ]:
from keras import initializers, models, regularizers
from keras.layers import Dense, Dropout, Embedding, SeparableConv1D, MaxPooling1D, GlobalAveragePooling1D

In [ ]:
toyModel = models.Sequential()
toyModel.add(Embedding(num_words, 
                    embedding_dim, 
                    input_length=max_len,
                      weights=[embedding_matrix],
                      trainable=False))